In [1]:
import os, shutil, json
import numpy as np
from tqdm import tqdm
from glob import glob

In [2]:
def read_json(path):
    with open(path, 'r') as json_file:
        json_data = json.load(json_file)
        
    return json_data

def get_split_by_video(path):
    videos = np.array([each.split('/')[-1].split('_')[0] for each in path])
    each_video = np.unique(videos)
    
    video_dict = {}
    for video_name in each_video:
        video_dict[video_name] = path[videos == video_name]
    return video_dict

def make_folder_v1(base_path, json_file, key):
    """
    base_path: our VFP290K directory
    """
    
    os.makedirs(base_path+key, exist_ok=True)
    
    
    for each_video in tqdm(json_file[key]['train'], total=len(json_file[key]['train'])):
        shutil.copytree(f'{base_path}/VFP290K/{each_video}', f'{base_path}/{key}/train/{each_video}')
        break
        
    for each_video in tqdm(json_file[key]['val'], total=len(json_file[key]['val'])):
        shutil.copytree(f'{base_path}/VFP290K/{each_video}', f'{base_path}/{key}/val/{each_video}')
        break
        
    for each_video in tqdm(json_file[key]['test'], total=len(json_file[key]['test'])):
        shutil.copytree(f'{base_path}/VFP290K/{each_video}', f'{base_path}/{key}/test/{each_video}')
        break
        
def make_folder_v2(base_path, json_file, key):
    """
    base_path: our VFP290K directory
    """
    
    img_paths = sorted([y for x in os.walk(base_path+'/VFP290K/') for y in glob(os.path.join(x[0], '*/images/*.jpg')) ])
    xml_paths = sorted([y for x in os.walk(base_path+'/VFP290K/') for y in glob(os.path.join(x[0], '*/clean_xml/*.xml')) ])

    img_dict = get_split_by_video(np.array(list(img_paths)))
    xml_dict = get_split_by_video(np.array(list(xml_paths)))
    
    for each_video in tqdm(json_file[key]['train'], total=len(json_file[key]['train'])):
        for each_image_file, each_xml_file in zip(img_dict[each_video], xml_dict[each_video]):
            os.makedirs(f'{base_path}/yolov5/{key}/train/image', exist_ok=True)
            os.makedirs(f'{base_path}/yolov5/{key}/train/label', exist_ok=True)
            
            shutil.copy(each_image_file, f'{base_path}/yolov5/{key}/train/image/')
            shutil.copy(each_xml_file, f'{base_path}/yolov5/{key}/train/label/')
        break
        
    for each_video in tqdm(json_file[key]['val'], total=len(json_file[key]['val'])):
        for each_image_file, each_xml_file in zip(img_dict[each_video], xml_dict[each_video]):    
            os.makedirs(f'{base_path}/yolov5/{key}/val/image', exist_ok=True)
            os.makedirs(f'{base_path}/yolov5/{key}/val/label', exist_ok=True)
            
            shutil.copy(each_image_file, f'{base_path}/yolov5/{key}/val/image/')
            shutil.copy(each_xml_file, f'{base_path}/yolov5/{key}/val/label/')
        break
        
    for each_video in tqdm(json_file[key]['test'], total=len(json_file[key]['test'])):
        for each_image_file, each_xml_file in zip(img_dict[each_video], xml_dict[each_video]):
            os.makedirs(f'{base_path}/yolov5/{key}/test/image', exist_ok=True)
            os.makedirs(f'{base_path}/yolov5/{key}/test/label', exist_ok=True)
            
            shutil.copy(each_image_file, f'{base_path}/yolov5/{key}/test/image/')
            shutil.copy(each_xml_file, f'{base_path}/yolov5/{key}/test/label/')
        break

### 1. Preparing Training Folder for baselines using MMdetection 

In [3]:
path   = '/media/data1/VFP290K/'
videos = sorted(os.listdir(path+'VFP290K'))

In [4]:
light_condition_data = read_json('light condition.json')
make_folder_v1(path, light_condition_data, 'day')
make_folder_v1(path, light_condition_data, 'night')

In [6]:
camera_hieght_data = read_json('camera angle.json')
make_folder_v1(path, camera_hieght_data, 'low')
make_folder_v1(path, camera_hieght_data, 'high')

  0%|          | 0/18 [00:01<?, ?it/s]


In [7]:
background_data = read_json('background.json')
make_folder_v1(path, background_data, 'street')
make_folder_v1(path, background_data, 'park')
make_folder_v1(path, background_data, 'building')

  0%|          | 0/4 [00:00<?, ?it/s]


In [4]:
benchmark_data = read_json('benchmark.json')
make_folder_v1(path, benchmark_data, 'benchmark')

  0%|          | 0/56 [00:00<?, ?it/s]


In [8]:
os.makedirs(path+'annotations', exist_ok=True)

### 2. Preparing Training Folder for YOLOv5

In [3]:
path   = '/media/data1/VFP290K/'
videos = sorted(os.listdir(path+'VFP290K'))

In [4]:
light_condition_data = read_json('light condition.json')
make_folder_v2(path, light_condition_data, 'day')
make_folder_v2(path, light_condition_data, 'night')

  0%|          | 0/8 [00:00<?, ?it/s]


In [5]:
camera_hieght_data = read_json('camera angle.json')
make_folder_v2(path, camera_hieght_data, 'low')
make_folder_v2(path, camera_hieght_data, 'high')

  0%|          | 0/18 [00:00<?, ?it/s]


In [6]:
background_data = read_json('background.json')
make_folder_v2(path, background_data, 'street')
make_folder_v2(path, background_data, 'park')
make_folder_v2(path, background_data, 'building')

  0%|          | 0/4 [00:00<?, ?it/s]


In [8]:
benchmark_data = read_json('benchmark.json')
make_folder_v2(path, benchmark_data, 'benchmark')

  0%|          | 0/56 [00:00<?, ?it/s]
